<a href="https://colab.research.google.com/github/jcandane/PhysicsI_Labs/blob/main/ndrcf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [ ]:
import numpy as np
from numpy import exp, sqrt

#### Plotting tools
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go


def plot2d(data = None):
    """ Plot in plotly
    """
    fig = go.Figure()
    #fig.add_trace(go.Surface(x=x, y=y, z=z))
    if data is not None:
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2], mode='markers'))

    fig.update_layout(title='Ground Truth Random-Contious 2d-Function', autosize=True, width=800, height=800, margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

def K(X, Y, ξ=10):
    """
    compute kernel function (RBF) between two domain points

    R_iax = X_ix - Y_ax
    Σ     = exp( - sum( R_iax**2 , over=x) / ξ )

    INPUT  : X (X data)
             Y (Y data)
             ξ (correlation length)
    RETURN : Σ
    """
    if X.ndim==1:
        X=X.reshape(-1,1)
    if Y.ndim==1:
        Y=Y.reshape(-1,1)
    return np.exp( - np.sum( (X[None, :] - Y[:, None])**2 , axis=2) / ξ )

def nK(R_ix, ξ=10):
    """
    compute kernel function (RBF) between two domain points

    R_iax = X_ix - Y_ax
    Σ     = exp( - sum( R_iax**2 , over=x) / ξ )

    INPUT  : X (X data)
             Y (Y data)
             ξ (correlation length)
    RETURN : Σ
    """
    R_ij = np.linalg.norm(R_ix[:, None, :] - R_ix[None, :, :], axis=2)
    return np.exp( - R_ij**2 / ξ )

def ndrcf(domain, dr_x, μ_i=None):
    """
    IN:
        domain : 2d-numpy.ndarray, gives ranges for each coordinate (coordinate-index 'x', 2), e.g. [[x0_min, x0_max], [x1_min, x1_max]]
        dr_x : the spacing for each dimension (has coordinate-index 'x')
        μ_i : mean along each point (points-index 'i')
    OUT:
        F_ix : 2d-numpy.ndarray (points-index, coordinate-index)

    **this should be made into an object where we save Σ_ij,
    perferbly as Σ_ij = L_ik L_kj (ie Cholesky-decomposition), save L_ij (lower-triangular-matrix)
    such that we may ask for additional points.
    """

    R_ix = np.stack(np.meshgrid(*[ np.arange(domain[i,0], domain[i,1], dr_x[i]) for i in range(len(dr_x)) ]), axis=-1)
    R_ix = R_ix.reshape((np.prod( R_ix.shape[:-1] ), R_ix.shape[-1])) ### x input

    μ_i  = np.zeros(R_ix.shape[0])
    Σ_ij = nK(R_ix)

    D_iy  = np.random.multivariate_normal(μ_i, Σ_ij, 1).T ### y output, #1 is the number of functions/surfaces
    return np.concatenate( (R_ix, D_iy) , axis=1)

# input parameters

In [ ]:
S    = 200 ## Number of Data Points
μ    = 0. ## Mean for all Data
ξ    = 15 ## correlation-length in covaraince matrix (sq of actual correlation-length ?)
show = 1 ## show these many samples

# 1d random-continous-function

In [ ]:
dr_x   = np.array([0.6])
domain = np.array([[ 0.0, 10.0]])

f = ndrcf(domain, dr_x, μ_i=None)
print(f.shape)

(17, 2)


# 2d random-continous-function

In [ ]:
dr_x   = np.array([0.6, 0.5])
domain = np.array([[ 0.0, 10.0],
                   [-5.0,  7.0]])

f = ndrcf(domain, dr_x, μ_i=None)
print(f.shape)

plot2d(data = f)

(408, 3)


# 3d random-continous-function

In [ ]:
dr_x   = np.array([0.8, 0.9, 0.2])
domain = np.array([[ 0.0, 10.0],
                   [-5.0,  7.0],
                   [ 2.0,  8.0]])

f = ndrcf(domain, dr_x, μ_i=None)
print(f.shape)

# 7d random-continous-function

In [ ]:
dr_x   = np.array([1.2, 1.75, 1.2, 2.4, 0.6, 0.8, 0.40])
domain = np.array([[ 0.0, 10.0],
                   [-5.0,  7.0],
                   [ 2.0,  4.0],
                   [ 2.0,  4.0],
                   [ 1.0,  3.0],
                   [11.0, 13.0],
                   [ 5.0,  6.0]])

f = ndrcf(domain, dr_x, μ_i=None)
print(f.shape)

(4536, 8)